# 重采样与频率转换

不同时间序列的频率是不同的，有些场景将一个时间序列的频率进行转换。重采样就是时间序列进行频率转换的过程。从高频到低频的转换过程称为向下采样，也称为降采样，反之，从低频到高频的转换称为向上采样，也称为升采样。向上采样的过程有些类似`groupby`的分组聚合操作，先分组在进行操作；向下采样并不需要聚合操作，而是插值操作。

本节通过实例来介绍常用的重采样和频率转换操作。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

## 股票交易数据集

本节示例会使用一个股票的交易数据，数据使用 sqlite 数据库存放，共有两个表`dailybar`与`min5bar`。下面来创建一个数据库连接：

In [ ]:
import os
dbfile = os.path.join('..', 'data', 'XSHG_000001.db')
url = 'sqlite:///{}'.format(dbfile)
dbfile, url

In [ ]:
import sqlalchemy

engine = sqlalchemy.create_engine(url)

使用 `pd.read_sql()`函数读取数据文件，并设置日期为索引：

In [ ]:
dailybar = pd.read_sql('dailybar', index_col=['date'], con=engine)
min5bar = pd.read_sql('min5bar', index_col=['barstop'], con=engine)
dailybar.info()

In [ ]:
# 5分钟K线
min5bar.info()

## 重采样

重采样是指将时间序列从一个频率转换为另一个频率：
- 向下采样：高频聚合到低频。
- 向上采样：从低频插值为高频。

Pandas 对象的重采样使用`resample()`方法，其使用语法为：
```python
dailybar.resample(rule, how=None, axis=0, fill_method=None, closed=None, label=None, convention='start', kind=None, loffset=None, limit=None, base=0, on=None, level=None)
```
主要参数
- `rule`，重采样频率偏置字符串或转换对象；
- `how`，聚合函数名或数组函数，已丢弃；
- `axis=0`,重采样方向；
- `fill_method`，向上采样的插值方法，`ffill`或`bfill`，缺省不差之；
- `closed`，向下采样时，时间间隔的开闭；
- `label`，向下采样时，如何标记聚合结果；
- `convention`，仅对`Period`对象，对区间重新采样时，将低频转为高频的约定；
- `loffset`，对标签进行时间调校；
- `kind`，转换类型；
- `limit`，向前或向后填充时，填充区间的最大值；
- `on`，对于`DataFrame`对象，指定时间列而非索引

`pd.resample()`对数据进行分组，返回类似`groupby`的对象，然后调用聚合函数，聚合函数有：
- `first`，首个数据
- `last`，最后数据
- `median`，返回中值
- `max`，最大值
- `min`，最小值

### 向下采样

`min5bar`是5分钟K线，一行数据在 5 分钟内交易的开盘价、最高价、最低价、收盘价、成家量与成交值，时刻为5分钟内的结束时间。也就是说`2017-03-15 09:35:00`表示从`09:30`到`09:34:59`的时间段，不包括`09:35`时刻。注意其中价格单位为分：

In [ ]:
df.head(10)

下面进行向下采样，时间频率降为每30分，重采样的频率设置为`30T`，其它使用缺省值，聚合函数使用`first()`：

In [ ]:
# 深拷贝
df = min5bar.copy()
df['open'].resample('30T').first().head(10)

可以看出，在分隔数据时缺省为`closed='left'`，即数据按照左闭右开来分隔；聚合数据结果缺省会使用`label='left`'，即标签索引使用间隔左边时间。下面更改参数，并使用聚合函数`max()`:

In [ ]:
df['high'].resample('30T', closed='right', label='right').max().head(10)

有上可知，在向下采样时，需要搞清楚如何选择时间间隔，然后聚合箱体如何表示。对这里的情况，时间间隔左闭右开，箱体标识则用间隔结束位置表示。

In [ ]:
# 分组 low 列，聚合函数为 min
df['low'].resample('30T', closed='left', label='right').min().head(10)

使用`apply()`方法可以指定聚合函数，在这个示例中，对如下列要使用不同聚合函数：
- 对`open`列要用聚合函数`first`
- 对`high`列要用聚合函数`max`
- 对`low`列要用聚合函数`min`
- 对`close`列要用聚合函数`last`
- 对`volumn`列要用聚合函数`sum`
- 对`amount`列要用聚合函数`last`。

可以使用字典来分别指定：

In [ ]:
ohlc_dict = {
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum',
    'amount': 'sum',
}

In [ ]:
newdf = df.resample('30T', closed='left', label='right').apply(ohlc_dict)
newdf.head(10)

最后把删除掉有缺失值的行即可：

In [ ]:
colnames = ['open', 'high', 'low', 'close', 'volume', 'amount']
newdf.dropna(inplace=True)
newdf.head(10)

### 向上采样与插值

时序数据从低频到高频转换时，不需要任何聚合。从地频到高频转换，其间隙会产生缺失值。可以使用如下方法
- `asfreq()`，转换到高频，产生缺失值。
- `ffill()`，向前填充缺失值。
- `bfill()`，向后填充缺失值。

下面首先拷贝日线数据，

In [ ]:
# 深拷贝
df = dailybar[dailybar.index > '2018-01-01'].copy()
df.info()

然后向上采样到`60T`，即每小时，分别使用`asfreq()`与`ffill()`来填充缺失值：

In [ ]:
# 转换到高频
newdf = df.resample('60T').asfreq()
newdf.head(10)

In [ ]:
# 使用前填充
newdf = df.resample('60T').ffill()
newdf.head(10)